# Google Maps Scraper
Bot that retrieves the data from a query to google maps by using the **Selenium** API

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import time
import re

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_py import binary_path

## Functions

In [2]:
def select_language():
    '''returns selected language'''   
    while True:
        try:
            language = int(input('\nPlease select language (English = 0, Espanol = 1): '))
            print('')
            
        except ValueError:
            continue
        
        if language in {0,1}:
            break
        else:
            continue
        
    return language

def query_google_maps(language_selection):
    '''input: query to be made
        return: url to get'''
    
    query1 = input(f'{language_selection[0]}: ')
    assert query1 != '','ERROR!: query cannot be blank'    
    query2 = input(f'{language_selection[1]}: ')
    assert query2 != '','ERROR!: query cannot be blank'    
    print('')
    query = query1+' '+query2
    query = query.replace(' ', '+')
    
    if int(language_selection[9]) == 0:
        url = "https://www.google.com/maps/search/"+query
    
    
    elif int(language_selection[9]) == 1:
        url = "https://www.google.cl/maps/search/"+query 
    
    return query,url
    
    
def load_all_results(driver):
    '''loads the complete list of the query results by scrolling down'''
    
    load_list_element= driver.find_element(By.XPATH,"//div[@class='qjESne veYFef']")    
    while True:
        try:
            time.sleep(2)
            scroll_down = ActionChains(driver).move_to_element(load_list_element).perform()
            
        except: #Last result
            break

def get_data(driver, sleep_time=3):
    '''returns a dictionary with the information of each result'''    
    time.sleep(sleep_time) 
    dict_data = {}
    
    #name
    name = driver.find_element(By.XPATH,"//h1[@class='DUwDvf fontHeadlineLarge']").text
    name = ''.join(re.findall('[\w\s\d]',name))  #to remove special characters  
    dict_data['name'] = name
    #rating
    try:
        rating = driver.find_element(By.XPATH,"//div[@class='LBgpqf']//div[@role='button']").text
        dict_data['rating'] = rating

    except:
        dict_data['rating'] = np.nan
    #category       
    try:  
        category = driver.find_element(By.XPATH,"//div[@class='fontBodyMedium']// \
                                   button[@jsaction='pane.rating.category']").text
        dict_data['category'] = category
    
    except:
        dict_data['category'] = np.nan
       
    #address, location, phone, website
    data = driver.find_elements(By.XPATH,"//div[@class='RcCsl fVHpi w4vB1d NOE9ve M0S7ae AG25L']")        
    for i in data:

        attribute = i.find_element(By.XPATH,"./button[1]").get_attribute("data-item-id")
        text = i.find_element(By.XPATH,"./button[1]").text

        if attribute == None:
            attribute = i.find_element(By.XPATH,"./a").get_attribute("data-item-id") #for website
            text = i.find_element(By.XPATH,"./a").get_attribute("href")
        
        attribute = attribute.replace(':',' ').split()[0] #for formatting the phone
        dict_data[attribute] = text
               
    #coordenates
    current_url = driver.current_url
    latitude    = current_url.split('!8m2!3d')[1]
    latitude    = float(latitude.split('!4d')[0])
    longitude   = current_url.split('!4d')[1]
    longitude   = float(longitude.split('!')[0])
    
    dict_data['latitude']  = latitude
    dict_data['longitude'] = longitude
    
    return dict_data

def generate_df(scrape_data):
    '''generates a DataFrame from the data scraped'''
    df = pd.DataFrame(scrape_data)
    
    df.rename(columns={'authority':'website', 'oloc':'location'}, inplace=True)
    df['name'] = df['name'].apply(lambda row: row.lower().capitalize())
    df['location'] = df['location'].apply(lambda row: ' '.join(str(row).split()[1:]))
    df['rating'] = pd.to_numeric(df['rating'])
    df.replace('',np.nan, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[['name', 'category', 'rating', 'phone', 'address', 'location', 'latitude', 'longitude', 'website']]
    
    return df


def language_dict(language):
    '''language dictionary for English and Spanish'''
    
    dictionary = np.array([('What are you looking for?, ex:supermarket', 'Que andas buscando? ej: supermercado'),
                 ('Where? ex: santiago chile', 'En qué lugar? ej: santiago chile'),
                  ('Loading list with the results...', 'Cargando lista con los resultados...'),
                  ('Done! \n', 'Listo! \n'),
                  ('Scraping the data...', 'Extrayendo los datos...'),
                  ('Results found', 'Resultados encontrados'),
                  ('Getting data from the result', 'Extrayendo data del resultado'),
                  ('Saving file...', 'Guardando archivo'),
                  ('Do you want to save all the results? (all = 0, otherwise select the amount: ex: Top 10 results = 10)',
                   'Quieres guardar todos los resultados? (todos = 0, o seleccione la cantidad: ej: Top 10 resultados = 10)'),
                    (0,1)])
    
    return dictionary[:,language]

## Main
In the following example the bot is going to retrieve all the results for the following query and save the top 50 results: <br>
- query: pizza restaurants in Vina del Mar, Chile

In [3]:
#language selection
language = select_language()
language_selection = language_dict(language)

#querying google maps
query, url = query_google_maps(language_selection)

# to supress the error messages/logs
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")   
options.add_experimental_option('excludeSwitches', ['enable-logging'])  
driver = webdriver.Chrome(options=options,executable_path=binary_path)

driver.get(url)
time.sleep(1)

#Loading the list with the results
print(language_selection[2])
load_all_results(driver) 
print(language_selection[3]) 

results_list = driver.find_elements(By.CLASS_NAME, "hfpxzc") 
len_list = len(results_list)

#Scraping the data for each result
scrape_data = []
print(language_selection[4])
print(f'{language_selection[5]}: {len_list} ')

try:
    filter_results = int(input(f'{language_selection[8]}: '))
    assert filter_results >= 0
    print('')

except Exception as exception:        
    print(f'ERROR, must be a number >= 0 : {exception}')
    raise

if filter_results != 0:
    results_list = results_list[:filter_results]

for index,result in enumerate(results_list,start=1):
    print(f'{language_selection[6]}: {index}')             
    try:
        result.click()

        element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH,"//h1[@class='DUwDvf fontHeadlineLarge']")))

    except ElementClickInterceptedException:
        driver.execute_script("arguments[0].click();", result)

    finally:      
            data = get_data(driver)
            scrape_data.append(data)

#Dataframe and saving the file as csv
df = generate_df(scrape_data)

query_file_name = query.replace('+', '_')    
print(f'\n{language_selection[7]}: {query_file_name}.csv')
df.to_csv(f'{query_file_name}.csv', header=True, index=False, encoding="latin1")


Please select language (English = 0, Espanol = 1): 0

What are you looking for?, ex:supermarket: pizza restaurant
Where? ex: santiago chile: vina del mar chile

Loading list with the results...
Done! 

Scraping the data...
Results found: 121 
Do you want to save all the results? (all = 0, otherwise select the amount: ex: Top 10 results = 10): 50

Getting data from the result: 1
Getting data from the result: 2
Getting data from the result: 3
Getting data from the result: 4
Getting data from the result: 5
Getting data from the result: 6
Getting data from the result: 7
Getting data from the result: 8
Getting data from the result: 9
Getting data from the result: 10
Getting data from the result: 11
Getting data from the result: 12
Getting data from the result: 13
Getting data from the result: 14
Getting data from the result: 15
Getting data from the result: 16
Getting data from the result: 17
Getting data from the result: 18
Getting data from the result: 19
Getting data from the result: 20

In [4]:
df.sort_values(by='rating', ascending=False)

,name,category,rating,phone,address,location,latitude,longitude,website
41,Trae pizza pizzería artesanal,Pizzería,5.0,9 9924 9849,"Diego Portales 684, Valparaíso, Viña del Mar 2...",Viña del Mar,-33.026595,-71.571074,https://traepizza.cl/
29,Pizzería uruguay,Pizzería,5.0,9 9215 7052,"Esmeralda 235, Viña del Mar, Valparaíso",Viña del Mar,-32.986708,-71.507378,https://www.instagram.com/pizzeriauruguay
33,Apetitus delivery viña del mar,Pizzería,4.8,(32) 335 9522,"Diego Portales 1190, Valparaíso, Viña del Mar ...",Viña del Mar,-33.028288,-71.574771,http://apetitus.cl/
48,Pizzabrosa a la piedra,Pizzería,4.8,9 5528 4745,"Viña del Mar, Valparaíso",Viña del Mar,-32.998209,-71.493570,NaN
18,El orégano pizzeria caffe cervecería resturant,Cafetería,4.7,(32) 312 1878,"3 Pte. 157, Viña del Mar, Valparaíso",Viña del Mar,-33.020440,-71.555060,http://www.eloregano.com/
34,Mozza pizza,Pizzería,4.7,9 3779 7083,"Von Schroeder 110, LOCAL 3, Viña del Mar, Valp...",Viña del Mar,-33.022773,-71.561255,https://wa.me/56937797083
13,Fusione pizza amici e,Pizzería,4.7,(32) 268 3969,"Av. Valparaíso 2, local D, Valparaíso, Viña de...",Viña del Mar,-33.024648,-71.562650,http://www.fusione.cl/
11,Badras,Pizzería,4.6,9 4201 0191,"5 Nte. 164, Viña del Mar, Valparaíso",Viña del Mar,-33.017546,-71.557533,http://www.facebook.com/badraschile
38,Piazza recreo,Pizzería,4.6,(32) 360 1509,"5 de Abril 110, Valparaíso, Viña del Mar 25200...",Viña del Mar,-33.028887,-71.579522,http://www.piazzarecreo.cl/
24,Casa reñaca restaurant,Restaurante italiano,4.6,9 6638 0478,"Vicuña Mackenna 901, Valparaíso, Viña del Mar ...",Viña del Mar,-32.972257,-71.533171,NaN
